In [ ]:
import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ['CUDA_VISIBLE_DEVICES']='-1'
os.environ['TF_CUDNN_USE_AUTOTUNE'] ='0'

import numpy as np
import random as rn
import tensorflow as tf

rn.seed(14)
np.random.seed(14)
tf.random.set_seed(14)

# from keras import backend as k
import tensorflow.keras.backend as k

config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1,
allow_soft_placement=True, device_count = {'CPU': 1})
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(),config=config)
tf.compat.v1.keras.backend.set_session(sess)

import pandas as pd
import datetime as dt

import keras
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, LSTM, Bidirectional
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from math import sqrt
from keras.callbacks import EarlyStopping
from keras.layers import Layer
import keras.backend as K
import keras_tuner as kt

import seaborn as sn
from matplotlib import style
import matplotlib.pyplot as plt

In [ ]:
from keras.models import Sequential
from tensorflow.keras.models import *

from keras.layers import Dense, Dropout,RepeatVector, Input,Flatten, Conv1D,MaxPooling1D
rn.seed(14)
np.random.seed(14)
tf.random.set_seed(14)
model = Sequential()


In [ ]:
# Cluster Datasets:
# Cluster 1: November, December, January, February
# Cluster 2: June, July, August
# Cluster 3: April, May, September, October, March

### Cluster 1

In [ ]:
# Cluster 1: November, December, January, February

In [ ]:
# Load your original dataset
df = pd.read_excel(r"UP08.xlsx", index_col='Date_Time')
df = df[['dissolvedoxygenmeasured']]
df=df['2017-04-01':'2021-03-31']

# Filter DataFrame for the months November, December, January, February
df = df[df.index.month.isin([11, 12, 1, 2])]

# Print or use the resulting DataFrame
print(df)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Assuming you have a time series data stored in a pandas DataFrame called 'data'
# with a column named 'value' containing the values

# Convert the 'value' column to a numpy array
# values = data['value'].to_numpy()

# Plot ACF
fig, ax = plt.subplots(figsize=(10, 3))
plot_acf(df['dissolvedoxygenmeasured'][:100], ax=ax,alpha=0.05)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function (ACF)')
plt.show()

# Plot PACF
fig, ax = plt.subplots(figsize=(10, 3))
plot_pacf(df['dissolvedoxygenmeasured'][:100], ax=ax,alpha=0.05)
plt.xlabel('Lag')
plt.ylabel('Partial Autocorrelation')
plt.title('Partial Autocorrelation Function (PACF)')
plt.show()

In [ ]:
df_train=df['2017-04-01':'2020-03-31']
df_test=df['2020-04-01':]
df_train.shape, df_test.shape

In [ ]:
# convert an array of values into a dataset matrix
import numpy 
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back+1, 0])
    return np.array(dataX), np.array(dataY)

def percentage_error(actual, predicted):
    res = numpy.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return numpy.mean(numpy.abs(percentage_error(numpy.asarray(y_true), numpy.asarray(y_pred)))) * 100


In [ ]:
datasets=df.values
X, Y = create_dataset(datasets, 9)

train_size = df_train.shape[0]
test_size =  df_test.shape[0]
trainX, testX = X[0:train_size], X[train_size:]
trainY, testY = Y[0:train_size], Y[train_size:]
X_train=pd.DataFrame(trainX)
Y_train=pd.DataFrame(trainY)
X_test=pd.DataFrame(testX)
Y_test=pd.DataFrame(testY)
sc_X = MinMaxScaler()
sc_y = MinMaxScaler()
X= sc_X.fit_transform(X_train)
y= sc_y.fit_transform(Y_train)
X1= sc_X.transform(X_test)
y1= sc_y.transform(Y_test)
y=y.ravel()
y1=y1.ravel()

In [ ]:
rn.seed(14)
np.random.seed(14)
tf.random.set_seed(14)

model = Sequential()
model.add(Dense(units = 32, activation='relu'))
model.add(Dense(16))
model.add(Dense(8))
model.add(Dense(4))
# model.add(Dense(3))
model.add(Dense(2))
model.add(Dense(1))
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mse',optimizer=opt)

es = tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=5,verbose=1,mode="min")
mcp_save = ModelCheckpoint('.UP08_c1_mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, epsilon=1e-4, mode='min')

model.fit(trainX, y, validation_split = 0.1, epochs = 1000, batch_size = 64,verbose=1
          , callbacks =[es, mcp_save, reduce_lr_loss]
         ) 


y_pred_train = model.predict(trainX)
y_pred_test = model.predict(testX)

y_pred_test= np.array(y_pred_test).ravel()
y_pred_test=pd.DataFrame(y_pred_test)

y1=pd.DataFrame(y1)
y=pd.DataFrame(y)                           

y_pred_train= np.array(y_pred_train).ravel()
y_pred_train=pd.DataFrame(y_pred_train)

y_test= sc_y.inverse_transform (y1)
y_train= sc_y.inverse_transform (y)
y_pred_test1= sc_y.inverse_transform (y_pred_test)
y_pred_train1= sc_y.inverse_transform (y_pred_train)      

mape_train1_c1= mean_absolute_percentage_error(y_train,y_pred_train1)
rmse_train1_c1= sqrt(mean_squared_error(y_train,y_pred_train1))
mae_train1_c1= mean_absolute_error(y_train,y_pred_train1)

mape_test1_c1= mean_absolute_percentage_error(y_test,y_pred_test1)
rmse_test1_c1= sqrt(mean_squared_error(y_test,y_pred_test1))
mae_test1_c1= mean_absolute_error(y_test,y_pred_test1)


print(" \n train results")
print(mape_train1_c1)
print(rmse_train1_c1)
print(mae_train1_c1)

print("test results")
print(mape_test1_c1)
print(rmse_test1_c1)
print(mae_test1_c1)


### Cluster 2

In [ ]:
# Cluster 2: June, July, August

In [ ]:
# Load your original dataset
df = pd.read_excel(r"UP08.xlsx", index_col='Date_Time')
df = df[['dissolvedoxygenmeasured']]
df=df['2017-04-01':'2021-03-31']

# Assuming you have already loaded and filtered your DataFrame df
# If not, you can use the provided code to load and filter it

# Filter DataFrame for the months 'June', 'July', 'August'
df = df[df.index.month.isin([6, 7, 8])]

# Print or use the resulting DataFrame
print(df)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Assuming you have a time series data stored in a pandas DataFrame called 'data'
# with a column named 'value' containing the values

# Convert the 'value' column to a numpy array
# values = data['value'].to_numpy()

# Plot ACF
fig, ax = plt.subplots(figsize=(10, 3))
plot_acf(df['dissolvedoxygenmeasured'][:100], ax=ax,alpha=0.05)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function (ACF)')
plt.show()

# Plot PACF
fig, ax = plt.subplots(figsize=(10, 3))
plot_pacf(df['dissolvedoxygenmeasured'][:100], ax=ax,alpha=0.05)
plt.xlabel('Lag')
plt.ylabel('Partial Autocorrelation')
plt.title('Partial Autocorrelation Function (PACF)')
plt.show()

In [ ]:
df_train=df['2017-04-01':'2020-03-31']
df_test=df['2020-04-01':]
df_train.shape, df_test.shape

In [ ]:
# convert an array of values into a dataset matrix
import numpy 
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back+1, 0])
    return np.array(dataX), np.array(dataY)

def percentage_error(actual, predicted):
    res = numpy.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return numpy.mean(numpy.abs(percentage_error(numpy.asarray(y_true), numpy.asarray(y_pred)))) * 100


In [ ]:
datasets=df.values
X, Y = create_dataset(datasets, 5)

train_size = df_train.shape[0]
test_size =  df_test.shape[0]
trainX, testX = X[0:train_size], X[train_size:]
trainY, testY = Y[0:train_size], Y[train_size:]
X_train=pd.DataFrame(trainX)
Y_train=pd.DataFrame(trainY)
X_test=pd.DataFrame(testX)
Y_test=pd.DataFrame(testY)
sc_X = MinMaxScaler()
sc_y = MinMaxScaler()
X= sc_X.fit_transform(X_train)
y= sc_y.fit_transform(Y_train)
X1= sc_X.transform(X_test)
y1= sc_y.transform(Y_test)
y=y.ravel()
y1=y1.ravel()

In [ ]:
rn.seed(14)
np.random.seed(14)
tf.random.set_seed(14)

model = Sequential()
model.add(Dense(units = 8, activation='relu'))
# model.add(Dense(16))
model.add(Dense(8))
model.add(Dense(4))
model.add(Dense(2))
model.add(Dense(1))
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mse',optimizer=opt)

es = tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=5,verbose=1,mode="min")
mcp_save = ModelCheckpoint('.UP08_c2_mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, epsilon=1e-4, mode='min')

model.fit(trainX, y, validation_split = 0.1, epochs = 1000, batch_size = 64,verbose=1
          , callbacks =[es, mcp_save, reduce_lr_loss]
         ) 

y_pred_train = model.predict(trainX)
y_pred_test = model.predict(testX)

y_pred_test= np.array(y_pred_test).ravel()
y_pred_test=pd.DataFrame(y_pred_test)

y1=pd.DataFrame(y1)
y=pd.DataFrame(y)                           

y_pred_train= np.array(y_pred_train).ravel()
y_pred_train=pd.DataFrame(y_pred_train)

y_test= sc_y.inverse_transform (y1)
y_train= sc_y.inverse_transform (y)
y_pred_test1= sc_y.inverse_transform (y_pred_test)
y_pred_train1= sc_y.inverse_transform (y_pred_train)      

mape_train1_c2= mean_absolute_percentage_error(y_train,y_pred_train1)
rmse_train1_c2= sqrt(mean_squared_error(y_train,y_pred_train1))
mae_train1_c2= mean_absolute_error(y_train,y_pred_train1)

mape_test1_c2= mean_absolute_percentage_error(y_test,y_pred_test1)
rmse_test1_c2= sqrt(mean_squared_error(y_test,y_pred_test1))
mae_test1_c2= mean_absolute_error(y_test,y_pred_test1)


print(" \n train results")
print(mape_train1_c2)
print(rmse_train1_c2)
print(mae_train1_c2)

print("test results")
print(mape_test1_c2)
print(rmse_test1_c2)
print(mae_test1_c2)


### Cluster 3

In [ ]:
# Cluster 3: April, May, September, October, March

In [ ]:
# Load your original dataset
df = pd.read_excel(r"UP08.xlsx", index_col='Date_Time')
df = df[['dissolvedoxygenmeasured']]
df=df['2017-04-01':'2021-03-31']


# Filter DataFrame for the months April, May, September, October, March

df = df[df.index.month.isin([3, 4, 5, 9, 10])]

# Print or use the resulting DataFrame
print(df)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Assuming you have a time series data stored in a pandas DataFrame called 'data'
# with a column named 'value' containing the values

# Convert the 'value' column to a numpy array
# values = data['value'].to_numpy()

# Plot ACF
fig, ax = plt.subplots(figsize=(10, 3))
plot_acf(df['dissolvedoxygenmeasured'][:100], ax=ax,alpha=0.05)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function (ACF)')
plt.show()

# Plot PACF
fig, ax = plt.subplots(figsize=(10, 3))
plot_pacf(df['dissolvedoxygenmeasured'][:100], ax=ax,alpha=0.05)
plt.xlabel('Lag')
plt.ylabel('Partial Autocorrelation')
plt.title('Partial Autocorrelation Function (PACF)')
plt.show()

In [ ]:
df_train=df['2017-04-01':'2020-03-31']
df_test=df['2020-04-01':]
df_train.shape, df_test.shape

In [ ]:
# convert an array of values into a dataset matrix
import numpy 
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back+1, 0])
    return np.array(dataX), np.array(dataY)

def percentage_error(actual, predicted):
    res = numpy.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return numpy.mean(numpy.abs(percentage_error(numpy.asarray(y_true), numpy.asarray(y_pred)))) * 100


In [ ]:
datasets=df.values
X, Y = create_dataset(datasets, 8)

train_size = df_train.shape[0]
test_size =  df_test.shape[0]
trainX, testX = X[0:train_size], X[train_size:]
trainY, testY = Y[0:train_size], Y[train_size:]
X_train=pd.DataFrame(trainX)
Y_train=pd.DataFrame(trainY)
X_test=pd.DataFrame(testX)
Y_test=pd.DataFrame(testY)
sc_X = MinMaxScaler()
sc_y = MinMaxScaler()
X= sc_X.fit_transform(X_train)
y= sc_y.fit_transform(Y_train)
X1= sc_X.transform(X_test)
y1= sc_y.transform(Y_test)
y=y.ravel()
y1=y1.ravel()

In [ ]:
rn.seed(14)
np.random.seed(14)
tf.random.set_seed(14)

model = Sequential()
model.add(Dense(units = 32, activation='relu'))
model.add(Dense(units = 16))
model.add(Dense(8))
model.add(Dense(4))
model.add(Dense(2))
model.add(Dense(1))
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mse',optimizer=opt)

es = tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=5,verbose=1,mode="min")
mcp_save = ModelCheckpoint('.UP08_c3_mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

model.fit(trainX, y, validation_split = 0.1, epochs = 1000, batch_size = 64,verbose=1
          , callbacks =[es, mcp_save, reduce_lr_loss]
         ) 


y_pred_train = model.predict(trainX)
y_pred_test = model.predict(testX)

y_pred_test= np.array(y_pred_test).ravel()
y_pred_test=pd.DataFrame(y_pred_test)

y1=pd.DataFrame(y1)
y=pd.DataFrame(y)                           

y_pred_train= np.array(y_pred_train).ravel()
y_pred_train=pd.DataFrame(y_pred_train)

y_test= sc_y.inverse_transform (y1)
y_train= sc_y.inverse_transform (y)
y_pred_test1= sc_y.inverse_transform (y_pred_test)
y_pred_train1= sc_y.inverse_transform (y_pred_train)      

mape_train1_c3= mean_absolute_percentage_error(y_train,y_pred_train1)
rmse_train1_c3= sqrt(mean_squared_error(y_train,y_pred_train1))
mae_train1_c3= mean_absolute_error(y_train,y_pred_train1)

mape_test1_c3= mean_absolute_percentage_error(y_test,y_pred_test1)
rmse_test1_c3= sqrt(mean_squared_error(y_test,y_pred_test1))
mae_test1_c3= mean_absolute_error(y_test,y_pred_test1)


print(" \n train results")
print(mape_train1_c3)
print(rmse_train1_c3)
print(mae_train1_c3)

print("test results")
print(mape_test1_c3)
print(rmse_test1_c3)
print(mae_test1_c3)


# whole

In [ ]:
# Load your original dataset
df = pd.read_excel(r"UP08.xlsx", index_col='Date_Time')
df = df[['dissolvedoxygenmeasured']]
df=df['2017-04-01':'2021-03-31']
df

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Assuming you have a time series data stored in a pandas DataFrame called 'data'
# with a column named 'value' containing the values

# Convert the 'value' column to a numpy array
# values = data['value'].to_numpy()

# Plot ACF
fig, ax = plt.subplots(figsize=(10, 3))
plot_acf(df['dissolvedoxygenmeasured'][:100], ax=ax,alpha=0.05)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function (ACF)')
plt.show()

# Plot PACF
fig, ax = plt.subplots(figsize=(10, 3))
plot_pacf(df['dissolvedoxygenmeasured'][:100], ax=ax,alpha=0.05)
plt.xlabel('Lag')
plt.ylabel('Partial Autocorrelation')
plt.title('Partial Autocorrelation Function (PACF)')
plt.show()

In [ ]:
# convert an array of values into a dataset matrix
import numpy 
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back+1, 0])
    return np.array(dataX), np.array(dataY)

def percentage_error(actual, predicted):
    res = numpy.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return numpy.mean(numpy.abs(percentage_error(numpy.asarray(y_true), numpy.asarray(y_pred)))) * 100


In [ ]:
# Split the data into training and testing sets
train_data = df.loc['2017-04-01':'2020-03-31']
test_data = df.loc['2020-04-01':]

In [ ]:
train_data.shape, test_data.shape

In [ ]:
datasets=df.values
X, Y = create_dataset(datasets, 4)

train_size = train_data.shape[0]
test_size =  test_data.shape[0]
trainX, testX = X[0:train_size], X[train_size:]
trainY, testY = Y[0:train_size], Y[train_size:]
X_train=pd.DataFrame(trainX)
Y_train=pd.DataFrame(trainY)
X_test=pd.DataFrame(testX)
Y_test=pd.DataFrame(testY)
sc_X = MinMaxScaler()
sc_y = MinMaxScaler()
X= sc_X.fit_transform(X_train)
y= sc_y.fit_transform(Y_train)
X1= sc_X.transform(X_test)
y1= sc_y.transform(Y_test)
y=y.ravel()
y1=y1.ravel()

In [ ]:
rn.seed(14)
np.random.seed(14)
tf.random.set_seed(14)

model = Sequential()
model.add(Dense(units = 16, activation='relu'))
model.add(Dense(units = 8))
model.add(Dense(4))
model.add(Dense(2))
model.add(Dense(1))
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mse',optimizer=opt)

es = tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=5,verbose=1,mode="min")
mcp_save = ModelCheckpoint('.UP08_mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, epsilon=1e-4, mode='min')

model.fit(trainX, y, validation_split = 0.1, epochs = 1000, batch_size = 64,verbose=1
          , callbacks =[es, mcp_save, reduce_lr_loss]
         ) 

y_pred_train = model.predict(trainX)
y_pred_test = model.predict(testX)

y_pred_test= np.array(y_pred_test).ravel()
y_pred_test=pd.DataFrame(y_pred_test)

y1=pd.DataFrame(y1)
y=pd.DataFrame(y)                           

y_pred_train= np.array(y_pred_train).ravel()
y_pred_train=pd.DataFrame(y_pred_train)

y_test= sc_y.inverse_transform (y1)
y_train= sc_y.inverse_transform (y)
y_pred_test1= sc_y.inverse_transform (y_pred_test)
y_pred_train1= sc_y.inverse_transform (y_pred_train)      

mape_train1= mean_absolute_percentage_error(y_train,y_pred_train1)
rmse_train1= sqrt(mean_squared_error(y_train,y_pred_train1))
mae_train1= mean_absolute_error(y_train,y_pred_train1)

mape_test1= mean_absolute_percentage_error(y_test,y_pred_test1)
rmse_test1= sqrt(mean_squared_error(y_test,y_pred_test1))
mae_test1= mean_absolute_error(y_test,y_pred_test1)


print(" \n train results")
print(mape_train1)
print(rmse_train1)
print(mae_train1)

print("test results")
print(mape_test1)
print(rmse_test1)
print(mae_test1)


In [ ]:
# Comparison

In [ ]:
# using whole dataset vs clusters for prediction 
print(mape_test1, mape_test1_c1,mape_test1_c2,mape_test1_c3)
print(rmse_test1, rmse_test1_c1,rmse_test1_c2,rmse_test1_c3)
print(mae_test1, mae_test1_c1,mae_test1_c2,mae_test1_c3)

print("\n")
# using whole dataset vs clusters for prediction 
print(mape_test1, (((4/12)*mape_test1_c1)+((3/12)*mape_test1_c2)+((5/12)*mape_test1_c3)))
print(rmse_test1, (((4/12)*rmse_test1_c1)+((3/12)*rmse_test1_c2)+((5/12)*rmse_test1_c3)))
print(mae_test1, (((4/12)*mae_test1_c1)+((3/12)*mae_test1_c2)+((5/12)*mae_test1_c3)))
